In [1]:
import sys
sys.version

'3.6.7 (v3.6.7:6ec5cf24b7, Oct 20 2018, 03:02:14) \n[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]'

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import cv2

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import to_categorical
from keras.models import load_model
from keras.constraints import max_norm
#from keras.wrappers.scikit_learn import KerasClassifier
%matplotlib inline

In [4]:
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten

In [ ]:
img = cv2.imread('/Users/anubhavjain/Desktop/sign-language-mnist/amer_sign2.png')

In [ ]:
df=pd.read_csv('/Users/anubhavjain/Desktop/sign-language-mnist/sign_mnist_train.csv')
d_test=pd.read_csv('/Users/anubhavjain/Desktop/sign-language-mnist/sign_mnist_test.csv')

In [ ]:
df.head()

In [ ]:
df.loc[0:2,:]

In [ ]:
t=np.array(df.loc[0,:])

In [ ]:
x_train=np.array(df[df.columns[df.columns!='label']])
x_label=np.array(df['label'])

In [ ]:
y_test=np.array(d_test[d_test.columns[d_test.columns!='label']])
y_label=np.array(d_test['label'])

In [ ]:
x_label=to_categorical(x_label)
y_label=to_categorical(y_label)

In [ ]:
#x_train.shape
#np.unique(y_label)
no_classes=y_label.shape[1]

In [ ]:
x_train.shape
#y_label.shape

In [ ]:
x_train=x_train.reshape(x_train.shape[0],28,28,1)
y_test=y_test.reshape(y_test.shape[0],28,28,1)

In [ ]:
x_train=x_train/255
y_test=y_test/255

In [ ]:
model=Sequential()

In [ ]:
model=load_model('gesture_to_text.h5')

In [ ]:
model.add(Conv2D(16,(5,5),strides=(1,1),activation='relu',input_shape=(28,28,1)))
model.add(MaxPooling2D((2,2),strides=(2,2)))

model.add(Conv2D(8,(2,2),strides=(1,1),activation='relu'))
model.add(MaxPooling2D((2,2),strides=(2,2)))
model.add(Flatten())

model.add(Dropout(0.2))
model.add(Dense(200,activation='relu',kernel_initializer='normal',kernel_constraint=max_norm(3)))
model.add(Dropout(0.4))
model.add(Dense(100,activation='relu',kernel_initializer='normal',kernel_constraint=max_norm(3)))
model.add(Dense(no_classes,activation='softmax'))

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(x_train,x_label,validation_data=(y_test,y_label),epochs=20,batch_size=19,verbose=2)

In [ ]:
t_before=time.time()
scores=model.evaluate(y_test,y_label,verbose=0)
t_after=time.time()

In [ ]:
scores[1]

In [ ]:
(t_after-t_before)

In [ ]:
model.save('gesture_to_text.h5')